In [1]:
from pathlib import Path
import sys

here = Path.cwd()
repo_root = next(p for p in [here, *here.parents] if (p/"pyproject.toml").exists())
src = repo_root / "src"
if str(src) not in sys.path:
    sys.path.insert(0, str(src))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from optimal_ipr.distributions import value_distribution, build_theta_distribution
from optimal_ipr.probability import build_subjective_probability
from optimal_ipr.cost import build_cost_function
from optimal_ipr.fee import build_fee_schedule
from optimal_ipr.lookup import build_lookup_tables
from optimal_ipr.outcomes import welfare_outcomes
from optimal_ipr.baseline import BaselineModel
from optimal_ipr.regulator import RegulatorModel
from optimal_ipr.government import GovernmentModel

In [3]:
# value distribution
test_n_v = 51
test_sigma = 3.0

v_grid, v_weights = value_distribution(n_v=test_n_v, sigma=test_sigma)

In [4]:
# theta distribution
f, F, F_inv = build_theta_distribution(noise_level=0.20)

# scalar wrapper for F when a scalar is required
F_scalar = lambda t: float(F(np.array([t])))

In [5]:
p = build_subjective_probability(base_k=1.2, m_comp=5, F=F, F_inv=F_inv)

In [6]:
c = build_cost_function(f, TARGET_AVG_COST_SHARE=0.50, C_MIN_COST=0.05, GAMMA_C_COST=3.0)

In [7]:
Z = build_fee_schedule(zeta=0.04, fee_M=1.5)

In [ ]:
tau_d_grid = np.array([0.15, 0.20, 0.25, 0.30])
tau_f_grid = np.array([0.0, 0.05, 0.10, 0.15])
bar_beta_grid = np.linspace(0.0, 1.0, 101)

reg_prefs = {
    "welfarist_balanced": {"phi": 0.6, "psi": 0.3},
    #"pro_innovation":     {"phi": 0.9, "psi": 0.1},
    #"public_good":        {"phi": 0.1, "psi": 0.9},
    #"utilitarian":        "special_case",
    #"rawlsian":           "special_case"
}

gov_prefs = {
    "utilitarian":          lambda th: 1.0,
    #"elitist":              lambda th: 1.0 + th,
    #"strongly_elitist":     lambda th: (1.0 + th)**2,
    #"egalitarian":          lambda th: 1.0 / (1.0 + th),
    #"strongly_egalitarian": lambda th: 1.0 / (1.0 + th)**2
}

In [9]:
theta_tilde_table, theta_winner_table, get_tau_d_index, get_tau_f_index, get_bar_beta_index, get_v_index = \
    build_lookup_tables(p, c, Z, f, F_scalar, tau_d_grid, tau_f_grid, bar_beta_grid, v_grid)

Processing tau pairs:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
results_table = welfare_outcomes(
    tau_d=tau_d_grid,
    tau_f=tau_f_grid,
    gov_prefs=gov_prefs,
    reg_prefs=reg_prefs,
    v_grid=v_grid,
    v_weights=v_weights,
    theta_tilde_table=theta_tilde_table,
    theta_winner_table=theta_winner_table,
    f=f,
    F=F,            # vectorized is fine; module wraps when needed
    F_inv=F_inv,
    p=p,
    c=c,
    Z=Z,
    feas=False,      # enforce feasibility
)

results_table

,Tau D,Tau F,Gov Pref,Reg Pref,Optimal Patent Breadth Cap,Expected Optimal Patent Breadth Granted,Welfare % Change,Innovator Welfare % Change,Imitator Welfare % Change,Non-Investor Welfare % Change
0,0.15,0.0,utilitarian,welfarist_balanced,0.94,0.94,1.123959,-6.205494,8.491012,113.999987
1,0.20,0.0,utilitarian,welfarist_balanced,0.88,0.88,2.227323,-12.424503,15.989818,227.999824


In [11]:
results_path = repo_root / "results"
if str(results_path) not in sys.path:
    sys.path.insert(0, str(results_path))

csv_path = results_path / "full_model_results.csv"
results_table.to_csv(csv_path, index=False)